导入相应的包

In [1]:
from keras.datasets import cifar10
import numpy as np
np.random.seed(22)

D:\annocoda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


准备数据

In [2]:
# 准备数据
(x_img_train,y_label_train),(x_img_test,y_label_test)=cifar10.load_data()
x_img_train_normalize = x_img_train.astype('float32') / 255.0
x_img_test_normalize = x_img_test.astype('float32') / 255.0
 
from keras.utils import np_utils
y_label_train_OneHot = np_utils.to_categorical(y_label_train)
y_label_test_OneHot = np_utils.to_categorical(y_label_test)

In [6]:
# 构造优化器
from keras.optimizers import Adam,SGD,Adamax
from keras.callbacks import CSVLogger,TensorBoard
sgd = SGD(lr=0.01)
adam = Adam()

Method1 从数据出发解决过拟合问题

In [3]:
# Mixup迭代实现
class MixupGenerator():
    def __init__(self, X_train, y_train, batch_size=32, alpha=0.2, shuffle=True, datagen=None):
        self.X_train = X_train
        self.y_train = y_train
        self.batch_size = batch_size
        self.alpha = alpha
        self.shuffle = shuffle
        self.sample_num = len(X_train)
        self.datagen = datagen

    def __call__(self):
        while True:
            indexes = self.__get_exploration_order()
            itr_num = int(len(indexes) // (self.batch_size * 2))

            for i in range(itr_num):
                batch_ids = indexes[i * self.batch_size * 2:(i + 1) * self.batch_size * 2]
                X, y = self.__data_generation(batch_ids)

                yield X, y

    def __get_exploration_order(self):
        indexes = np.arange(self.sample_num)

        if self.shuffle:
            np.random.shuffle(indexes)

        return indexes

    def __data_generation(self, batch_ids):
        _, h, w, c = self.X_train.shape
        l = np.random.beta(self.alpha, self.alpha, self.batch_size)
        X_l = l.reshape(self.batch_size, 1, 1, 1)
        y_l = l.reshape(self.batch_size, 1)

        X1 = self.X_train[batch_ids[:self.batch_size]]
        X2 = self.X_train[batch_ids[self.batch_size:]]
        X = X1 * X_l + X2 * (1 - X_l)

        if self.datagen:
            for i in range(self.batch_size):
                X[i] = self.datagen.random_transform(X[i])
                X[i] = self.datagen.standardize(X[i])

        if isinstance(self.y_train, list):
            y = []

            for y_train_ in self.y_train:
                y1 = y_train_[batch_ids[:self.batch_size]]
                y2 = y_train_[batch_ids[self.batch_size:]]
                y.append(y1 * y_l + y2 * (1 - y_l))
        else:
            y1 = self.y_train[batch_ids[:self.batch_size]]
            y2 = self.y_train[batch_ids[self.batch_size:]]
            y = y1 * y_l + y2 * (1 - y_l)

        return X, y

In [ ]:
#构建图像处理的迭代器
from keras.preprocessing.image import ImageDataGenerator
 
#图片生成器
noa_gen = ImageDataGenerator()
simple_gen = ImageDataGenerator(rotation_range=40,
                              width_shift_range=0.2,
                              height_shift_range=0.2,
                              horizontal_flip=True,
                              fill_mode='nearest')
noa_it = noa_gen.flow(x=x_img_train_normalize,y=y_label_train_OneHot,batch_size=32)
simple_it = simple_gen.flow(x=x_img_train_normalize,y=y_label_train_OneHot,batch_size=32)
mixup_it = MixupGenerator(x_img_train_normalize,y_label_train_OneHot)()
iterator_dict = {"noa":noa_it,
                "simplea":simple_it,
                "mixup":mixup_it}

In [13]:
from keras.losses import categorical_crossentropy 
for name,method in iterator_dict.items():
    csvlogger = CSVLogger("data/"+name+".csv")
    tensorboard = TensorBoard(log_dir="overfit/"+name+"/")
    model = load_normal()
    model.compile(loss='categorical_crossentropy',optimizer=Adamax(), metrics=['accuracy'])
    train_history = model.fit_generator(generator=method,
                        validation_data = (x_img_test_normalize,y_label_test_OneHot),
                        epochs=100, 
                        steps_per_epoch=len(x_img_train_normalize)//32,
                        callbacks=[csvlogger,tensorboard]) 

Epoch 1/100
1562/1562 [==============================] - 26s 16ms/step - loss: 1.4509 - acc: 0.4829 - val_loss: 1.1820 - val_acc: 0.5829
Epoch 2/100
1562/1562 [==============================] - 26s 16ms/step - loss: 1.0920 - acc: 0.6156 - val_loss: 1.0329 - val_acc: 0.6410cc: - ETA: 7s - loss: 1.1108 - acc: 0.60 - ETA: 7s - loss: 1.1100 - acc - ETA: 7s - loss: 1.1096 - acc - ETA: 6s - loss: 1.1075 -  - ETA: 5s - loss: 1.1061 - ETA: 5s - loss: 1.1034 - ac - ETA: 4s - loss: 1.1025 - acc: 0.6 - ETA: 4s - loss: 1.1014  - ETA: 3s - loss: 1 - ETA: 0s - loss: 1.0919 - acc: 0
Epoch 3/100
1562/1562 [==============================] - 26s 16ms/step - loss: 0.9446 - acc: 0.6685 - val_loss: 0.9370 - val_acc: 0.6774 - ETA: 8s - loss: 0.9583 - ETA: 7s - loss: 0.9554 - acc:  - ETA: 7s - loss: 0. -  - ETA: 4s -  - ETA: 3s - loss: 0.9467 - acc: 0.6 - ETA: 2s - loss: 0.9467 -
Epoch 4/100
1562/1562 [==============================] - 26s 17ms/step - loss: 0.8488 - acc: 0.7026 - val_loss: 0.9069 - val_acc: 

1562/1562 [==============================] - 25s 16ms/step - loss: 0.0032 - acc: 0.9993 - val_loss: 2.7139 - val_acc: 0.7054 los - ETA: 3s
Epoch 76/100
1562/1562 [==============================] - 25s 16ms/step - loss: 3.5103e-04 - acc: 1.0000 - val_loss: 2.6967 - val_acc: 0.7129s - loss: 4.0409e-04 -  - ETA: 5s - loss: 3.9556e- - ETA: 4s 
Epoch 77/100
1562/1562 [==============================] - 26s 16ms/step - loss: 0.0033 - acc: 0.9992 - val_loss: 2.7121 - val_acc: 0.7068 ETA: 14s - loss - ETA: 13 - E
Epoch 78/100
1562/1562 [==============================] - 25s 16ms/step - loss: 0.0026 - acc: 0.9993 - val_loss: 2.7286 - val_acc: 0.7084151e-04 - acc:  - ETA: 4 - ETA: 0s - loss: 0.0025 - acc
Epoch 79/100
1562/1562 [==============================] - 26s 17ms/step - loss: 0.0039 - acc: 0.9989 - val_loss: 2.7450 - val_acc: 0.7072 0.0045 - ac - ETA: 7s - loss: 0.004 - ETA: 7 - ETA: 5s - loss: 0.0040 - acc: 0.9 - ETA: 5s - loss: 0.0040 - acc: 0 - ETA: 4s - los - ETA: 3s - loss: 0.0037 - E

1562/1562 [==============================] - 29s 19ms/step - loss: 0.7955 - acc: 0.7212 - val_loss: 0.8048 - val_acc: 0.7306
Epoch 67/100
1562/1562 [==============================] - 29s 18ms/step - loss: 0.7976 - acc: 0.7193 - val_loss: 0.7647 - val_acc: 0.7414: 5s - ETA: 4s -  - ETA: 2s - loss: 0.7974 - ac - ETA: 2s - ETA: 0s - loss: 0.7971 - acc:
Epoch 68/100
1562/1562 [==============================] - 29s 18ms/step - loss: 0.7931 - acc: 0.7223 - val_loss: 0.8265 - val_acc: 0.7221 - loss: 0.7922 - acc: 0 - ETA: 4s - lo - ETA: 2s -  - ETA: 1s - lo
Epoch 69/100
1562/1562 [==============================] - 29s 18ms/step - loss: 0.7941 - acc: 0.7219 - val_loss: 0.7550 - val_acc: 0.7466 ETA: 4s - loss: 0.7965 - ac -
Epoch 70/100
1562/1562 [==============================] - 29s 18ms/step - loss: 0.7877 - acc: 0.7231 - val_loss: 0.8204 - val_acc: 0.7207TA: 1s - los
Epoch 71/100
1562/1562 [==============================] - 29s 18ms/step - loss: 0.7878 - acc: 0.7237 - val_loss: 0.8005 - val

1562/1562 [==============================] - 27s 17ms/step - loss: 0.5182 - acc: 0.9477 - val_loss: 0.9586 - val_acc: 0.7117ss: 0.5176 - acc:  - ETA - ETA: 11 - ETA: 4s - loss: 0.5180 - acc: 0. - ETA:  - ETA: 2s - ETA: 0s - loss: 0.5181 - acc:
Epoch 46/100
1562/1562 [==============================] - 27s 17ms/step - loss: 0.5166 - acc: 0.9476 - val_loss: 0.9591 - val_acc: 0.7114cc:  - ETA: 7s - loss:  - ETA: 4s - loss: 0.5158 - acc: - ETA
Epoch 47/100
1562/1562 [==============================] - 27s 17ms/step - loss: 0.5111 - acc: 0.9474 - val_loss: 0.9748 - val_acc: 0.71176s - loss: 0.5097 - acc:  - ETA: 6s - loss: 0. - ETA: 5s - loss: 0.5102 - acc: 0 - ETA: 4s - loss: 0 - ETA: 3s -  - ETA: 2s - loss: 0.5101 - acc: 0.947 
Epoch 48/100
1562/1562 [==============================] - 27s 17ms/step - loss: 0.5155 - acc: 0.9470 - val_loss: 0.9865 - val_acc: 0.7042
Epoch 49/100
1562/1562 [==============================] - 30s 19ms/step - loss: 0.5102 - acc: 0.9488 - val_loss: 0.9627 - val_acc

Method2 从模型角度出发解决过拟合问题

In [5]:
# 构造模型
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten,BatchNormalization
from keras import regularizers

def load_normal():
    model = Sequential()
    model.add(Conv2D(filters=32,kernel_size=(3,3),
                     input_shape=(32, 32,3),
                     activation='relu', 
                     padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=64, kernel_size=(3, 3),
                     activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    return model

# def load_batchNormalization():
#     model = Sequential()
#     model.add(Conv2D(filters=32,kernel_size=(3,3),
#                      input_shape=(32, 32,3),
#                      activation='relu', 
#                      padding='same'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Conv2D(filters=64, kernel_size=(3, 3),
#                      activation='relu', padding='same'))
#     model.add(BatchNormalization())
#     model.add(MaxPooling2D(pool_size=(2, 2)))
#     model.add(Flatten())
#     model.add(Dense(128, activation='relu'))
#     model.add(Dense(10, activation='softmax'))
#     return model

def load_dropout01():
    model = Sequential()
    model.add(Conv2D(filters=32,kernel_size=(3,3),
                     input_shape=(32, 32,3),
                     activation='relu', 
                     padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=64, kernel_size=(3, 3),
                     activation='relu', padding='same'))
    model.add(Dropout(0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(10, activation='softmax'))
    return model

def load_dropout025():
    model = Sequential()
    model.add(Conv2D(filters=32,kernel_size=(3,3),
                     input_shape=(32, 32,3),
                     activation='relu', 
                     padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=64, kernel_size=(3, 3),
                     activation='relu', padding='same'))
    model.add(Dropout(0.25))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dropout(0.25))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(10, activation='softmax'))
    return model

def load_l2normalization025():
    model = Sequential()
    model.add(Conv2D(filters=32,kernel_size=(3,3),
                     input_shape=(32, 32,3),
                     kernel_regularizer=regularizers.l2(0.25),
                     activation='relu', 
                     padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=64, kernel_size=(3, 3),
                     kernel_regularizer=regularizers.l2(0.25),
                     activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    return model

def load_l2normalization01():
    model = Sequential()
    model.add(Conv2D(filters=32,kernel_size=(3,3),
                     input_shape=(32, 32,3),
                     kernel_regularizer=regularizers.l2(0.1),
                     activation='relu', 
                     padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=64, kernel_size=(3, 3),
                     kernel_regularizer=regularizers.l2(0.1),
                     activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    return model

def load_l1normalization01():
    model = Sequential()
    model.add(Conv2D(filters=32,kernel_size=(3,3),
                     input_shape=(32, 32,3),
                     kernel_regularizer=regularizers.l1(0.1),
                     activation='relu', 
                     padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=64, kernel_size=(3, 3),
                     kernel_regularizer=regularizers.l1(0.1),
                     activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    return model

def load_l1normalization025():
    model = Sequential()
    model.add(Conv2D(filters=32,kernel_size=(3,3),
                     input_shape=(32, 32,3),
                     kernel_regularizer=regularizers.l1(0.25),
                     activation='relu', 
                     padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=64, kernel_size=(3, 3),
                     kernel_regularizer=regularizers.l1(0.25),
                     activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    return model

# models_dict = {"normal":load_normal,
models_dict = {
               "l1normalization-0.1":load_l1normalization01,
                "l1normalization-0.25":load_l1normalization025,
                "l2normalization-0.1":load_l2normalization01,
                "l2normalization-0.25":load_l2normalization025,
                "dropout-0.1":load_dropout01,
                "dropout-0.25":load_dropout025}

In [ ]:
from keras.losses import categorical_crossentropy 
for name,model in models_dict.items():
    csvlogger = CSVLogger("new/"+name+".csv")
    tensorboard = TensorBoard(log_dir="new/"+name+"/")
    model = models_dict[name]()
    model.compile(loss='categorical_crossentropy',optimizer=adamax, metrics=['accuracy'])
    train_history = model.fit(x_img_train_normalize, y_label_train_OneHot,
                        validation_data = (x_img_test_normalize,y_label_test_OneHot),
                        epochs=100, 
                        batch_size=64,
                        callbacks=[csvlogger,tensorboard]) 